In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time

from IPython.display import display, HTML
from notebook.services.config import ConfigManager

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cf_columns = ['Make id', 'Make name', 'Model id', 'model name', 'Variant id', 'Variant name', 'Fuel', 'CC', 'Full']
cf_df = pd.DataFrame( columns = cf_columns )
cf_make = set()
cf_make_model = set()

cf_makes = (requests.get('https://www.coverfox.com/motor/fourwheeler/api/vehicle-make/').json())['data']

for make in cf_makes:
    models = (requests.get('https://www.coverfox.com/motor/fourwheeler/make/%s/' % make['id'] ).json())['data']
    
    if len( models ) == 0:
        print( make['name'], len( models ) )  # 
        
    for model in models:
        cf_make_model.add( "%s %s" % (make, model) )
        
        if len(models[model]['variants']) == 0:
            print( "%s %s" % (make, model), len(models[model]['variants'])  )
            
        for variant in models[model]['variants']:
            full = "%s %s %s %s %s" % ( str(make['name']), str(models[model]['name']), str(variant['name']), str(variant['fuel_type']), str(variant['cc']))
            cf_df.loc[len(cf_df.index)] = [ make['id'], make['name'], models[model]['id'], models[model]['name'], variant['id'], variant['name'], variant['cc'], variant['fuel_type'], full ]

Aston Martin 0
Bugatti 0
Ferrari 0
Hero 0
Jawa 0
Lamborghini 0
Opel 0


In [3]:
cf_df.to_excel("./cf_master_web.xlsx", index=False)

In [3]:
pb_columns = ["MakeId", "ModelId", "Make", "Model", "Variant", "FuelType", "TransmissionType"]
fuel_type = { 7: "petrol" , 4: "diesel" , 3: "cng", 5: "lpg", 9: "electric" }
transmission = { 0: "Manual", 1:"Automatic" }
pb_df = pd.DataFrame( columns = pb_columns )
pb_make = set()
pb_make_model = set()

headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Mobile Safari/537.36'}
pb_models = (requests.get('https://ci.policybazaar.com/js/MakeModel.json?v=626', headers=headers ).json())

count = 0

for model in pb_models:
    pb_make.add( model["MakeName"] )
    pb_make_model.add( model["ModelName"].replace(",", "")  )
#     try:
    variants = ( requests.get("https://ci.policybazaar.com/api/master/getallvariant?modelid=%s" % model["ModelId"], headers=headers ).json())
#     except ConnectionError, MaxRetryError, NewConnectionError:
#         print( model["MakeName"], model["ModelName"], "time out" )
#     print( model["ModelId"] )
#     if len( variants ) == 0:
#         print( model["ModelName"], len( variants ) )
    
    for variant in variants:
        pb_make_model.add("%s" % (model["ModelName"].replace(",", "")))
        
        if count == 600:
            print("sleep")
            time.sleep( 30 )
            print("awake")
            count = 0
#         if variant["TransmissionType"] != 1 :
#             print( model["ModelName"], " variant  ",  variant)
                
        pb_df.loc[len(pb_df.index)] = [ 
            model["MakeId"],
            model["ModelId"],
            model["MakeName"],
            model["ModelNameV2"],
            variant["VariantName"],
            fuel_type[variant["FuelTypeId"]],
            transmission[variant["TransmissionType"]],
        ] 
            
        count += 1

        

sleep
awake
sleep
awake


ConnectTimeout: HTTPSConnectionPool(host='ci.policybazaar.com', port=443): Max retries exceeded with url: /api/master/getallvariant?modelid=125 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fab125bf580>, 'Connection to ci.policybazaar.com timed out. (connect timeout=None)'))

In [ ]:
pb_df["Model"] = pb_df["Model"].str.replace(",", "")
# pb_df.to_excel("./pb_master_web.xlsx", index=False)
# pb_df

In [14]:
pb_models = (requests.get('https://ci.policybazaar.com/js/MakeModel.json?v=611', headers=headers ).json())

pb_models


[{'ModelId': 2,
  'ModelName': 'ASTON MARTIN, DB9',
  'AliasModelName': 'ASTON MARTIN DB9',
  'MakeId': 2,
  'ShowingOrder': None,
  'MakeName': 'ASTON MARTIN',
  'ModelNameV2': 'DB9',
  'IsTopMake': False,
  'IsTopModel': True,
  'IsPopular': False,
  'IsTopMakeNewCar': False,
  'ShowingOrderNewCar': 0,
  'IsTopModelNewCar': False,
  'IsPopularNewCar': False},
 {'ModelId': 3,
  'ModelName': 'ASTON MARTIN, DBS',
  'AliasModelName': 'ASTON MARTIN DBS',
  'MakeId': 2,
  'ShowingOrder': None,
  'MakeName': 'ASTON MARTIN',
  'ModelNameV2': 'DBS',
  'IsTopMake': False,
  'IsTopModel': True,
  'IsPopular': False,
  'IsTopMakeNewCar': False,
  'ShowingOrderNewCar': 0,
  'IsTopModelNewCar': False,
  'IsPopularNewCar': False},
 {'ModelId': 4,
  'ModelName': 'ASTON MARTIN, RAPIDE',
  'AliasModelName': 'ASTON MARTIN RAPIDE',
  'MakeId': 2,
  'ShowingOrder': None,
  'MakeName': 'ASTON MARTIN',
  'ModelNameV2': 'RAPIDE',
  'IsTopMake': False,
  'IsTopModel': True,
  'IsPopular': False,
  'IsTopMake

In [18]:
variants = requests.get("https://ci.policybazaar.com/api/master/getallvariant?modelid=3", headers=headers)
variants


<Response [200]>